In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import accuracy_score, classification_report
pd.set_option('display.max_columns',None)
import joblib

In [2]:
price_data = pd.read_parquet(f'C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Dados_bolsa_interpolar.parquet')
price_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker
0,2000-01-05,0.520882,0.520882,0.520882,0.520882,0.279469,985,ABEV3.SA
1,2000-01-06,0.494478,0.494478,0.494478,0.494478,0.265302,227,ABEV3.SA
2,2000-01-12,0.481293,0.481293,0.481293,0.481293,0.258228,1137,ABEV3.SA
3,2000-01-13,0.484589,0.484589,0.484589,0.484589,0.259996,606,ABEV3.SA
4,2000-01-14,0.494478,0.494478,0.494478,0.494478,0.265302,6445,ABEV3.SA


In [3]:
import numpy as np
import pandas as pd

# Transformar e ordenar os dados
price_data['Date'] = pd.to_datetime(price_data['Date'])
price_data.sort_values(by=['Ticker', 'Date'], inplace=True)

# Criar a coluna de mudança de preço
price_data['change_in_price'] = price_data['Close'].diff()

# Máscara para identificar a virada entre os tickers
mask = price_data['Ticker'] != price_data['Ticker'].shift(1)

# Função para suavização exponencial
def exponential_smoothing(data, alpha):
    smoothed = np.zeros(len(data))
    smoothed[0] = data[0]
    for t in range(1, len(data)):
        smoothed[t] = alpha * data[t] + (1 - alpha) * smoothed[t-1]
    return smoothed

# Função para calcular o target
def calculate_target(data, d):
    target = np.sign(data.shift(-d) - data)
    target[target == 0] = -1
    return target

# Função para calcular o OBV
def obv(group, smoothed_col):
    Volume = group['Volume']
    change = group[smoothed_col].diff()

    prev_obv = 0
    obv_values = []

    for i, j in zip(change, Volume):
        if i > 0:
            current_obv = prev_obv + j
        elif i < 0:
            current_obv = prev_obv - j
        else:
            current_obv = prev_obv

        prev_obv = current_obv
        obv_values.append(current_obv)

    return pd.Series(obv_values, index=group.index)

# Lista de valores alpha
alpha_list = [0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90]

# Período para calcular o target, RSI e estocástico
d = 10
n = 14
e = 9

# Loop para aplicar a suavização exponencial, calcular o target, o RSI e o estocástico
for alpha in alpha_list:
    col_name_smoothed = f"Smoothed_Close_{alpha}"
    col_name_prediction = f"Prediction_{alpha}"
    col_name_k_percent = f"k_percent_{alpha}"
    col_name_macd = f"MACD_{alpha}"
    col_name_macd_ema = f"MACD_EMA_{alpha}"
    col_name_price_rate_of_change = f"Price_Rate_Of_Change_{alpha}"
    col_name_obv = f"OBV_{alpha}"
    col_name_r_percent = f"r_percent_{alpha}"

    # Suavização exponencial
    price_data[col_name_smoothed] = price_data.groupby('Ticker')['Close'].transform(
        lambda x: exponential_smoothing(x.values, alpha)
    )

    # Aplicando a máscara para evitar cálculos na virada dos tickers
    price_data[col_name_smoothed] = np.where(mask, np.nan, price_data[col_name_smoothed])

    # Cálculo do target
    price_data[col_name_prediction] = price_data.groupby('Ticker')[col_name_smoothed].transform(
        lambda x: calculate_target(x, d)
    )
    price_data[col_name_prediction] = np.where(mask, np.nan, price_data[col_name_prediction])

    # Calculando o OBV usando a suavização exponencial
    obv_groups = price_data.groupby('Ticker').apply(obv, smoothed_col=col_name_smoothed)
    price_data[col_name_obv] = obv_groups.reset_index(level=0, drop=True)

    # Calculando o Price Rate of Change (ROC)
    price_data[col_name_price_rate_of_change] = price_data.groupby('Ticker')[col_name_smoothed].transform(lambda x: x.pct_change(periods=e))
    price_data[col_name_price_rate_of_change] = np.where(mask, np.nan, price_data[col_name_price_rate_of_change])

    # Calcular low_14 e high_14 para o estocástico
    low_14 = price_data.groupby('Ticker')['Low'].transform(lambda x: x.rolling(window=n).min())
    high_14 = price_data.groupby('Ticker')['High'].transform(lambda x: x.rolling(window=n).max())

    # Adicionar low_14 e high_14 ao DataFrame
    price_data['low_14'] = low_14
    price_data['high_14'] = high_14

    # Calcular o %K para o estocástico
    price_data[col_name_k_percent] = 100 * ((price_data[col_name_smoothed] - low_14) / (high_14 - low_14))

    # Aplicando a máscara para evitar cálculos na virada dos tickers
    price_data[col_name_k_percent] = np.where(mask, np.nan, price_data[col_name_k_percent])

    # Cálculo do MACD
    ema_26 = price_data.groupby('Ticker')[col_name_smoothed].transform(lambda x: x.ewm(span=26).mean())
    ema_12 = price_data.groupby('Ticker')[col_name_smoothed].transform(lambda x: x.ewm(span=12).mean())
    macd = ema_12 - ema_26

    # Cálculo da EMA do MACD
    ema_9_macd = macd.ewm(span=9).mean()

    # Adicionar MACD e MACD_EMA ao DataFrame
    price_data[col_name_macd] = macd
    price_data[col_name_macd_ema] = ema_9_macd

    # Cálculo do r_percent
    r_percent = ((high_14 - price_data[col_name_smoothed]) / (high_14 - low_14)) * (-100)
    price_data[col_name_r_percent] = r_percent

# Coluna de verificação (sem suavização, alpha = 1)
alpha = 1
price_data['Smoothed_Close_1'] = price_data.groupby('Ticker')['Close'].transform(
    lambda x: exponential_smoothing(x.values, alpha)
)
price_data['Smoothed_Close_1'] = np.where(mask, np.nan, price_data['Smoothed_Close_1'])

price_data['Verify'] = price_data.groupby('Ticker')['Smoothed_Close_1'].transform(
    lambda x: calculate_target(x, d)
)
price_data['Verify'] = np.where(mask, np.nan, price_data['Verify'])

# Cálculo do RSI
up_df = price_data[['Ticker', 'change_in_price']].copy()
down_df = price_data[['Ticker', 'change_in_price']].copy()

up_df['change_in_price'] = up_df['change_in_price'].where(up_df['change_in_price'] > 0, 0)
down_df['change_in_price'] = down_df['change_in_price'].where(down_df['change_in_price'] < 0, 0).abs()

ewma_up = up_df.groupby('Ticker')['change_in_price'].transform(lambda x: x.ewm(span=n).mean())
ewma_down = down_df.groupby('Ticker')['change_in_price'].transform(lambda x: x.ewm(span=n).mean())

relative_strength = ewma_up / ewma_down
price_data['RSI'] = 100.0 - (100.0 / (1.0 + relative_strength))

price_data['up_days'] = up_df['change_in_price']
price_data['down_days'] = down_df['change_in_price']

# Remover as linhas com NaN
price_data = price_data.dropna()

# Visualizar o DataFrame atualizado
price_data.tail()


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close_0.1,Prediction_0.1,OBV_0.1,Price_Rate_Of_Change_0.1,low_14,high_14,k_percent_0.1,MACD_0.1,MACD_EMA_0.1,r_percent_0.1,Smoothed_Close_0.2,Prediction_0.2,OBV_0.2,Price_Rate_Of_Change_0.2,k_percent_0.2,MACD_0.2,MACD_EMA_0.2,r_percent_0.2,Smoothed_Close_0.3,Prediction_0.3,OBV_0.3,Price_Rate_Of_Change_0.3,k_percent_0.3,MACD_0.3,MACD_EMA_0.3,r_percent_0.3,Smoothed_Close_0.4,Prediction_0.4,OBV_0.4,Price_Rate_Of_Change_0.4,k_percent_0.4,MACD_0.4,MACD_EMA_0.4,r_percent_0.4,Smoothed_Close_0.5,Prediction_0.5,OBV_0.5,Price_Rate_Of_Change_0.5,k_percent_0.5,MACD_0.5,MACD_EMA_0.5,r_percent_0.5,Smoothed_Close_0.6,Prediction_0.6,OBV_0.6,Price_Rate_Of_Change_0.6,k_percent_0.6,MACD_0.6,MACD_EMA_0.6,r_percent_0.6,Smoothed_Close_0.7,Prediction_0.7,OBV_0.7,Price_Rate_Of_Change_0.7,k_percent_0.7,MACD_0.7,MACD_EMA_0.7,r_percent_0.7,Smoothed_Close_0.8,Prediction_0.8,OBV_0.8,Price_Rate_Of_Change_0.8,k_percent_0.8,MACD_0.8,MACD_EMA_0.8,r_percent_0.8,Smoothed_Close_0.9,Prediction_0.9,OBV_0.9,Price_Rate_Of_Change_0.9,k_percent_0.9,MACD_0.9,MACD_EMA_0.9,r_percent_0.9,Smoothed_Close_1,Verify,RSI,up_days,down_days
11733,2024-12-06,57.470001,57.660000,56.750000,56.810001,56.307430,17897900,VALE3.SA,-0.989998,58.398757,-1.0,2782399148,-0.014726,56.750000,59.380001,62.690345,-0.600272,-0.481521,-37.309655,57.826427,-1.0,2376633870,-0.008835,40.928759,-0.748711,-0.705009,-59.071241,57.634482,-1.0,501570062,-0.006831,33.630485,-0.747428,-0.750508,-66.369515,57.480824,-1.0,907212466,-0.008759,27.787970,-0.736258,-0.756184,-72.212030,57.337344,-1.0,835109226,-0.011928,22.332445,-0.732294,-0.754184,-77.667555,57.209422,-1.0,445671538,-0.015003,17.468515,-0.733827,-0.751688,-82.531485,57.097973,-1.0,591627042,-0.017616,13.230922,-0.738109,-0.749984,-86.769078,56.998922,-1.0,-1010413342,-0.019789,9.464730,-0.743480,-0.749060,-90.535270,56.905484,-1.0,-844702038,-0.021662,5.911921,-0.749194,-0.748698,-94.088079,56.810001,-1.0,35.760334,0.00,0.989998
11734,2024-12-09,58.500000,60.189999,58.369999,59.830002,59.300713,40440600,VALE3.SA,3.020000,58.541881,-1.0,2822839748,-0.009233,56.750000,60.189999,52.089592,-0.595490,-0.504315,-47.910408,58.227142,-1.0,2417074470,0.001163,42.940186,-0.709459,-0.705899,-57.059814,58.293138,-1.0,542010662,0.007649,44.858682,-0.688934,-0.738193,-55.141318,58.420495,-1.0,947653066,0.011343,48.560923,-0.662733,-0.737494,-51.439077,58.583673,-1.0,875549826,0.014792,53.304460,-0.642370,-0.731821,-46.695540,58.781770,-1.0,486112138,0.018919,59.063104,-0.624999,-0.726350,-40.936896,59.010393,-1.0,632067642,0.023818,65.709134,-0.608146,-0.721616,-34.290866,59.263786,-1.0,-969972742,0.029351,73.075203,-0.590654,-0.717379,-26.924797,59.537550,-1.0,-804261438,0.035372,81.033463,-0.572122,-0.713382,-18.966537,59.830002,-1.0,61.366753,3.02,0.000000
11735,2024-12-10,59.740002,59.939999,59.290001,59.770000,59.241245,13089300,VALE3.SA,-0.060001,58.664693,-1.0,2835929048,-0.005543,56.750000,60.189999,55.659707,-0.575160,-0.518484,-44.340293,58.535714,-1.0,2430163770,0.006570,51.910298,-0.646004,-0.693920,-48.089702,58.736197,-1.0,555099962,0.013839,57.738301,-0.599910,-0.710536,-42.261699,58.960297,-1.0,960742366,0.018117,64.252852,-0.554515,-0.700898,-35.747148,59.176837,-1.0,888639126,0.021525,70.547603,-0.517279,-0.688913,-29.452397,59.374708,-1.0,499201438,0.024512,76.299689,-0.485313,-0.678143,-23.700311,59.542118,-1.0,645156942,0.026902,81.166262,-0.456975,-0.668688,-18.833738,59.668758,-1.0,-956883442,0.028439,84.847637,-0.431881,-0.660279,-15.152363,59.746755,-1.0,-791172138,0.028922,87.115018,-0.410182,-0.652742,-12.884982,59.770000,-1.0,60.811066,0.00,0.060001
11736,2024-12-11,59.500000,59.590000,58.450001,58.840000,58.319469,30138500,VALE3.SA,-0.930000,58.682224,-1.0,2866067548,-0.002775,56.750000,60.189999,56.169321,-0.551279,-0.525043,-43.830679,58.596571,-1.0,2460302270,0.009783,53.679407,-0.584073,-0.671950,-46.320593,58.767338,-1.0,585238462,0.016505,58.643564,

#### Se quiser rodar para um Ticker.

In [10]:
price_data = price_data[price_data['Ticker'].isin(['ABEV3.SA'])]
price_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close_0.93,Prediction_0.93,OBV_0.93,Price_Rate_Of_Change_0.93,low_14,high_14,k_percent_0.93,MACD_0.93,MACD_EMA_0.93,r_percent_0.93,Smoothed_Close_0.94,Prediction_0.94,OBV_0.94,Price_Rate_Of_Change_0.94,k_percent_0.94,MACD_0.94,MACD_EMA_0.94,r_percent_0.94,Smoothed_Close_0.95,Prediction_0.95,OBV_0.95,Price_Rate_Of_Change_0.95,k_percent_0.95,MACD_0.95,MACD_EMA_0.95,r_percent_0.95,Smoothed_Close_0.96,Prediction_0.96,OBV_0.96,Price_Rate_Of_Change_0.96,k_percent_0.96,MACD_0.96,MACD_EMA_0.96,r_percent_0.96,Smoothed_Close_0.97,Prediction_0.97,OBV_0.97,Price_Rate_Of_Change_0.97,k_percent_0.97,MACD_0.97,MACD_EMA_0.97,r_percent_0.97,Smoothed_Close_0.98,Prediction_0.98,OBV_0.98,Price_Rate_Of_Change_0.98,k_percent_0.98,MACD_0.98,MACD_EMA_0.98,r_percent_0.98,Smoothed_Close_0.99,Prediction_0.99,OBV_0.99,Price_Rate_Of_Change_0.99,k_percent_0.99,MACD_0.99,MACD_EMA_0.99,r_percent_0.99,Smoothed_Close_1.0,Prediction_1.0,OBV_1.0,Price_Rate_Of_Change_1.0,k_percent_1.0,MACD_1.0,MACD_EMA_1.0,r_percent_1.0,Smoothed_Close_1,Verify,RSI,up_days,down_days
13,2000-02-22,0.485255,0.485255,0.485255,0.485255,0.255174,75,ABEV3.SA,-0.029034,0.487228,1.0,26087,-0.013258,0.468107,0.527474,32.208622,0.000220,0.000340,-67.791378,0.486953,1.0,26087,-0.014017,31.745182,0.000217,0.000347,-68.254818,0.486676,1.0,26087,-0.014780,31.278115,0.000214,0.000354,-68.721885,0.486396,1.0,26087,-0.015547,30.807338,0.000211,0.000361,-69.192662,0.486115,1.0,26087,-0.016318,30.332747,0.000207,0.000367,-69.667253,0.485831,1.0,26087,-0.017092,29.854217,0.000203,0.000374,-70.145783,0.485544,1.0,26087,-0.017870,29.371599,0.000199,0.000380,-70.628401,0.485255,1.0,30637,-0.018652,28.884728,0.000194,0.000386,-71.115272,0.485255,1.0,43.244009,0.000000,0.029034
14,2000-02-23,0.494478,0.494478,0.494478,0.494478,0.260024,455,ABEV3.SA,0.009223,0.493971,1.0,26542,-0.059310,0.468107,0.527474,43.565460,-0.000044,0.000260,-56.434540,0.494026,1.0,26542,-0.059818,43.659770,-0.000042,0.000266,-56.340230,0.494088,1.0,26542,-0.060311,43.763168,-0.000039,0.000272,-56.236832,0.494155,1.0,26542,-0.060789,43.875758,-0.000037,0.000278,-56.124242,0.494227,1.0,26542,-0.061253,43.997650,-0.000034,0.000283,-56.002350,0.494305,1.0,26542,-0.061702,44.128955,-0.000031,0.000289,-55.871045,0.494389,1.0,26542,-0.062136,44.269789,-0.000029,0.000294,-55.730211,0.494478,1.0,31092,-0.062555,44.420276,-0.000026,0.000300,-55.579724,0.494478,1.0,47.299662,0.009223,0.000000
15,2000-02-24,0.487885,0.487885,0.487885,0.487885,0.256557,5005,ABEV3.SA,-0.006593,0.488311,1.0,21537,-0.068542,0.468107,0.527474,34.032339,-0.000579,0.000086,-65.967661,0.488253,1.0,21537,-0.068671,33.935491,-0.000579,0.000091,-66.064509,0.488195,1.0,21537,-0.068790,33.837211,-0.000580,0.000095,-66.162789,0.488136,1.0,21537,-0.068899,33.737231,-0.000580,0.000100,-66.262769,0.488075,1.0,21537,-0.068997,33.635278,-0.000581,0.000104,-66.364722,0.488013,1.0,21537,-0.069087,33.531076,-0.000581,0.000109,-66.468924,0.487950,1.0,21537,-0.069167,33.424343,-0.000582,0.000113,-66.575657,0.487885,1.0,26087,-0.069238,33.314793,-0.000583,0.000117,-66.685207,0.487885,1.0,44.667003,0.000000,0.006593
16,2000-02-25,0.474700,0.474700,0.474700,0.474700,0.249623,3033,ABEV3.SA,-0.013185,0.475653,1.0,18504,-0.053849,0.468107,0.527474,12.710363,-0.001745,-0.000291,-87.289637,0.475513,1.0,18504,-0.053692,12.475284,-0.001752,-0.000289,-87.524716,0.475375,1.0,18504,-0.053532,12.242070,-0.001760,-0.000287,-87.757930,0.475237,1.0,18504,-0.053369,12.010753,-0.001767,-0.000284,-87.989247,0.475101,1.0,18504,-0.053203,11.781377,-0.001775,-0.000283,-88.218623,0.474966,1.0,18504,-0.053034,11.553995,-0.001782,-0.000281,-88.446005,0.474833,1.0,18504,-0.052864,11.328672,-0.001790,-0.000279,-88.671328,0.474700,1.0,23054,-0.052692,11.105483,-0.001797,-0.000277,-88.894517,0.474700,1.0,39.583155,0.000000,0.013185
17,2000-02-29,0.468107,0.468107,0.468107,0.468107,0.246157,11602,ABEV3.SA,-0.006593,0.468635,1.0,6902,-0.0040

#### Se quiser normal só pular etapa anterior.

In [4]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Para cada alpha, criar as variáveis X_Cols e Y_Cols e treinar o modelo
for alpha in alpha_list:
    # Definindo as colunas de X e Y dinamicamente
    X_Cols = price_data[['RSI',
                         f'k_percent_{alpha}',
                         f'r_percent_{alpha}',
                         f'Price_Rate_Of_Change_{alpha}',
                         f'MACD_{alpha}',
                         f'MACD_EMA_{alpha}',
                         f'OBV_{alpha}']]
    
    Y_Cols = price_data[f'Prediction_{alpha}']

    # Dividir os dados em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X_Cols, Y_Cols, random_state=0)

    # Criar o classificador Random Forest
    rand_frst_clf = RandomForestClassifier(n_estimators=100, oob_score=True, criterion="gini", random_state=0)

    # Ajustar o modelo
    rand_frst_clf.fit(X_train, y_train)

    # Fazer previsões
    y_pred = rand_frst_clf.predict(X_test)

    # Imprimir a acurácia
    accuracy = accuracy_score(y_test, y_pred, normalize=True) * 100.0
    print(f'Alpha: {alpha}')
    print(f'Correct Prediction (%): {accuracy:.2f}')
    print('---')


Alpha: 0.1
Correct Prediction (%): 79.91
---
Alpha: 0.2
Correct Prediction (%): 72.06
---
Alpha: 0.3
Correct Prediction (%): 69.94
---
Alpha: 0.4
Correct Prediction (%): 66.20
---
Alpha: 0.5
Correct Prediction (%): 65.43
---
Alpha: 0.6
Correct Prediction (%): 65.13
---
Alpha: 0.7
Correct Prediction (%): 64.30
---
Alpha: 0.8
Correct Prediction (%): 65.03
---
Alpha: 0.9
Correct Prediction (%): 64.88
---


In [5]:
import warnings

# Desativa todos os warnings
warnings.filterwarnings("ignore")

# Para cada alpha, selecionar as colunas de X, gerar previsões e comparar com a coluna 'Verify'
for alpha in alpha_list:
    # Selecionar as colunas de X como um array NumPy
    X_Cols = price_data[['RSI',
                         f'k_percent_{alpha}',
                         f'r_percent_{alpha}',
                         f'Price_Rate_Of_Change_{alpha}',
                         f'MACD_{alpha}',
                         f'MACD_EMA_{alpha}',
                         f'OBV_{alpha}']].values

    # Gerar previsões e atualizar a coluna 'Prediction_{alpha}'
    price_data[f'Prediction_{alpha}'] = rand_frst_clf.predict(X_Cols)

    # Comparar as previsões de 'Prediction_{alpha}' com 'Verify' e calcular a precisão
    price_data['Match'] = (price_data[f'Prediction_{alpha}'] == price_data['Verify']).astype(int)  
    accuracy = price_data['Match'].mean()  # Proporção de acertos (iguais / total)

    # Imprimir a precisão para cada alpha
    print(f'\nPrecisão para Alpha: {alpha}')
    print(f'Precisão do modelo: {accuracy:.2%}')



Precisão para Alpha: 0.1
Precisão do modelo: 52.10%

Precisão para Alpha: 0.2
Precisão do modelo: 54.37%

Precisão para Alpha: 0.3
Precisão do modelo: 57.12%

Precisão para Alpha: 0.4
Precisão do modelo: 60.02%

Precisão para Alpha: 0.5
Precisão do modelo: 63.60%

Precisão para Alpha: 0.6
Precisão do modelo: 66.81%

Precisão para Alpha: 0.7
Precisão do modelo: 70.74%

Precisão para Alpha: 0.8
Precisão do modelo: 75.16%

Precisão para Alpha: 0.9
Precisão do modelo: 90.12%


In [6]:
import warnings
from datetime import timedelta

# Desativa todos os warnings
warnings.filterwarnings("ignore")

# Filtrar últimos 30 dias
last_30_days = price_data[price_data['Date'] >= price_data['Date'].max() - timedelta(days=30)]

# Para cada alpha, selecionar as colunas de X, gerar previsões e comparar com a coluna 'Verify'
for alpha in alpha_list:
    # Selecionar as colunas de X como um array NumPy
    X_Cols = last_30_days[['RSI',
                           f'k_percent_{alpha}',
                           f'r_percent_{alpha}',
                           f'Price_Rate_Of_Change_{alpha}',
                           f'MACD_{alpha}',
                           f'MACD_EMA_{alpha}',
                           f'OBV_{alpha}']].values

    # Gerar previsões e atualizar a coluna 'Prediction_{alpha}' para os últimos 30 dias
    last_30_days[f'Prediction_{alpha}'] = rand_frst_clf.predict(X_Cols)

    # Comparar as previsões de 'Prediction_{alpha}' com 'Verify' e calcular a precisão
    last_30_days['Match'] = (last_30_days[f'Prediction_{alpha}'] == last_30_days['Verify']).astype(int)  # 1 para igual, 0 para diferente
    accuracy = last_30_days['Match'].mean()  # Proporção de acertos (iguais / total)

    # Imprimir a precisão para cada alpha
    print(f'\nPrecisão para Alpha: {alpha}')
    print(f'Precisão do modelo nos últimos 30 dias: {accuracy:.2%}')



Precisão para Alpha: 0.1
Precisão do modelo nos últimos 30 dias: 40.00%

Precisão para Alpha: 0.2
Precisão do modelo nos últimos 30 dias: 48.57%

Precisão para Alpha: 0.3
Precisão do modelo nos últimos 30 dias: 42.86%

Precisão para Alpha: 0.4
Precisão do modelo nos últimos 30 dias: 42.86%

Precisão para Alpha: 0.5
Precisão do modelo nos últimos 30 dias: 53.33%

Precisão para Alpha: 0.6
Precisão do modelo nos últimos 30 dias: 69.52%

Precisão para Alpha: 0.7
Precisão do modelo nos últimos 30 dias: 63.81%

Precisão para Alpha: 0.8
Precisão do modelo nos últimos 30 dias: 74.29%

Precisão para Alpha: 0.9
Precisão do modelo nos últimos 30 dias: 90.48%


#### Salvar modelo 

In [8]:
# Salvar o modelo treinado
joblib.dump(rand_frst_clf, 'C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/random_forest_model.pkl')

['C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/random_forest_model.pkl']